<a href="https://colab.research.google.com/github/tonyw54/GeorgeBrown/blob/main/Computer_Vision2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


Kaggle credentials set.
Kaggle credentials successfully validated.


In [ ]:
import zipfile
import shutil
import os

# Re-download competition data
computer_vision_xm_path = kagglehub.competition_download('computer-vision-xm')

source_dir = '/root/.cache/kagglehub/competitions/computer-vision-xm'
destination_dir = '/content/computer-vision-xm'  # Create a folder in '/content/'

shutil.copytree(source_dir, destination_dir)

shutil.rmtree('/root/.cache/kagglehub/competitions/computer-vision-xm', ignore_errors=True)

print(f"Directory '{source_dir}' copied to '{destination_dir}' successfully.")

100%|██████████| 6.20G/6.20G [05:11<00:00, 21.4MB/s]

Extracting files...


Directory '/root/.cache/kagglehub/competitions/computer-vision-xm' copied to '/content/computer-vision-xm' successfully.


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
import timm
from PIL import Image
import pandas as pd
import os
from sklearn.model_selection import train_test_split
import numpy as np
import albumentations as A
from albumentations.pytorch import ToTensorV2

/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 1.4.21 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [ ]:
class LeafDataset(Dataset):
    def __init__(self, img_dir, df, transform=None, is_test=False):
        self.img_dir = img_dir
        self.df = df
        self.transform = transform
        self.is_test = is_test

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_name = self.df.iloc[idx, 1]
        img_path = os.path.join(self.img_dir, img_name)

        # Load image as numpy array for albumentations
        image = np.array(Image.open(img_path).convert('RGB'))

        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']

        if self.is_test:
            return image, img_name
        else:
            label = self.df.iloc[idx, 2]
            return image, label

In [ ]:
class ImprovedLeafClassifier(nn.Module):
    def __init__(self, num_classes=2):
        super(ImprovedLeafClassifier, self).__init__()
        # Use EfficientNetV2 as backbone - stronger than ResNet18
        self.model = timm.create_model('efficientnetv2_rw_s', pretrained=True)

        # Freeze early layers
        for name, param in self.model.named_parameters():
            if "blocks.0" in name or "blocks.1" in name or "blocks.2" in name:
                param.requires_grad = False

        # Replace classifier head
        num_features = self.model.classifier.in_features
        self.model.classifier = nn.Sequential(
            nn.Linear(num_features, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        return self.model(x)


In [ ]:
def create_data_loaders(img_dir, train_csv, test_csv, batch_size=16):
    train_df = pd.read_csv(train_csv)
    test_df = pd.read_csv(test_csv)

    # More aggressive data augmentation
    train_transform = A.Compose([
        A.RandomResizedCrop(height=384, width=384, scale=(0.8, 1.0)),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.RandomRotate90(p=0.5),
        A.ShiftScaleRotate(p=0.5),
        A.OneOf([
            A.GaussNoise(var_limit=(10.0, 50.0)),
            A.GaussianBlur(),
            A.MotionBlur(),
        ], p=0.3),
        A.OneOf([
            A.OpticalDistortion(),
            A.GridDistortion(),
        ], p=0.3),
        A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1, p=0.5),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2()
    ])

    val_transform = A.Compose([
        A.Resize(384, 384),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2()
    ])

    # Stratified split with larger validation set
    train_df, val_df = train_test_split(
        train_df, test_size=0.15, random_state=42, stratify=train_df['Labels']
    )

    # Create datasets
    train_dataset = LeafDataset(img_dir, train_df, train_transform)
    val_dataset = LeafDataset(img_dir, val_df, val_transform)
    test_dataset = LeafDataset(img_dir, test_df, val_transform, is_test=True)

    # Create data loaders with appropriate batch sizes
    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=4,
        pin_memory=True
    )
    val_loader = DataLoader(
        val_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=4,
        pin_memory=True
    )
    test_loader = DataLoader(
        test_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=4,
        pin_memory=True
    )

    return train_loader, val_loader, test_loader

In [ ]:
def train_model(model, train_loader, val_loader, num_epochs=20):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    # Use weighted loss if classes are imbalanced
    criterion = nn.CrossEntropyLoss()

    # Use AdamW with weight decay
    optimizer = optim.AdamW(
        [
            {'params': (p for n, p in model.named_parameters() if 'classifier' not in n), 'lr': 1e-4},
            {'params': (p for n, p in model.named_parameters() if 'classifier' in n), 'lr': 1e-3}
        ],
        weight_decay=0.01
    )

    # Cosine annealing scheduler with warmup
    scheduler = optim.lr_scheduler.OneCycleLR(
        optimizer,
        max_lr=[1e-4, 1e-3],
        epochs=num_epochs,
        steps_per_epoch=len(train_loader),
        pct_start=0.1,
        anneal_strategy='cos'
    )

    best_val_loss = float('inf')
    patience = 5
    patience_counter = 0

    scaler = torch.cuda.amp.GradScaler()  # For mixed precision training

    for epoch in range(num_epochs):
        # Training phase
        model.train()
        train_loss = 0
        correct = 0
        total = 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()

            # Use mixed precision training
            with torch.cuda.amp.autocast():
                outputs = model(images)
                loss = criterion(outputs, labels)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            scheduler.step()

            train_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

        train_accuracy = 100 * correct / total

        # Validation phase
        model.eval()
        val_loss = 0
        correct = 0
        total = 0

        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)

                val_loss += loss.item()
                _, predicted = outputs.max(1)
                total += labels.size(0)
                correct += predicted.eq(labels).sum().item()

        val_accuracy = 100 * correct / total
        avg_train_loss = train_loss / len(train_loader)
        avg_val_loss = val_loss / len(val_loader)

        print(f'Epoch [{epoch+1}/{num_epochs}]')
        print(f'Train Loss: {avg_train_loss:.4f}, Train Acc: {train_accuracy:.2f}%')
        print(f'Val Loss: {avg_val_loss:.4f}, Val Acc: {val_accuracy:.2f}%')

        # Early stopping with patience
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            patience_counter = 0
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'val_loss': best_val_loss,
            }, 'best_leaf_classifier.pth')
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f'Early stopping triggered after epoch {epoch+1}')
                break

In [ ]:
def generate_test_predictions(model, test_loader, output_file='submission.csv'):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    model.eval()

    predictions = []
    filenames = []

    with torch.no_grad():
        for images, img_names in test_loader:
            images = images.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)

            predictions.extend(predicted.cpu().numpy())
            filenames.extend(img_names)

    # Create predictions DataFrame
    predictions_df = pd.DataFrame({
        'Images': filenames,
        'Labels': predictions
    })

    # Save predictions
    predictions_df.to_csv(output_file, index=False)
    print(f"Predictions saved to {output_file}")

In [ ]:
# Usage example
if __name__ == "__main__":
    # Initialize model and data loaders
    model = ImprovedLeafClassifier()

    # Paths to your data
    IMG_DIR = '/content/computer-vision-xm/images/kaggle/working/Reorganized_Data/images'
    TRAIN_CSV = '/content/computer-vision-xm/train.csv'
    TEST_CSV = '/content/computer-vision-xm/test.csv'

    # Create data loaders
    train_loader, val_loader, test_loader = create_data_loaders(
        IMG_DIR, TRAIN_CSV, TEST_CSV, batch_size=32
    )

    # Train the model
    train_model(model, train_loader, val_loader)

    # Generate predictions for test set
    generate_test_predictions(model, test_loader)

<ipython-input-28-764f2e62a640>:31: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()  # For mixed precision training
<ipython-input-28-764f2e62a640>:46: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


Epoch [1/20]
Train Loss: 0.5472, Train Acc: 69.16%
Val Loss: 0.2756, Val Acc: 88.61%
Epoch [2/20]
Train Loss: 0.2163, Train Acc: 91.52%
Val Loss: 0.1573, Val Acc: 94.70%
Epoch [3/20]
Train Loss: 0.1563, Train Acc: 94.48%
Val Loss: 0.1302, Val Acc: 96.07%
Epoch [4/20]
Train Loss: 0.1225, Train Acc: 95.52%
Val Loss: 0.1200, Val Acc: 96.07%
Epoch [5/20]
Train Loss: 0.1046, Train Acc: 96.42%
Val Loss: 0.1043, Val Acc: 95.87%
Epoch [6/20]
Train Loss: 0.0940, Train Acc: 97.08%
Val Loss: 0.0823, Val Acc: 97.25%
Epoch [7/20]
Train Loss: 0.0835, Train Acc: 97.08%
Val Loss: 0.0854, Val Acc: 97.64%
Epoch [8/20]
Train Loss: 0.0691, Train Acc: 97.71%
Val Loss: 0.0794, Val Acc: 97.05%
Epoch [9/20]
Train Loss: 0.0646, Train Acc: 97.67%
Val Loss: 0.0997, Val Acc: 96.46%
Epoch [10/20]
Train Loss: 0.0690, Train Acc: 97.85%
Val Loss: 0.0904, Val Acc: 97.45%
Epoch [11/20]
Train Loss: 0.0544, Train Acc: 98.30%
Val Loss: 0.0945, Val Acc: 97.05%
Epoch [12/20]
Train Loss: 0.0433, Train Acc: 98.44%
Val Loss: 0